## Simulating encounters

If this is your first time using a notebook, please ensure you have [Python installed](https://www.python.org/downloads/) and you have installed the additional dependencies with the following command: `pip install -r ./automation/requirements.txt`

First, we'll navigate to the repo's root:

In [2]:
import os

if os.path.basename(os.getcwd()) != "TheGame":
    os.chdir("..")

Next, import what you need:

In [3]:
import copy
from automation.templates import Bestiary
from automation.simulator import Deck, Encounter

Use the Bestiary to provide dictionaries for each of the combatants. We can use emojis to represent them in the combat output. The Name value will be shown in the log.

Note that files with SAMPLE in the name are designed to workshop ideas and test edge cases, not be included in the game.

In [4]:
b = Bestiary(input_files="06_Bestiary_SAMPLE.yaml").raw_data
c1 = b["Clubs1"]
c1.update(dict(Name="💀", id="A"))
c2 = copy.copy(b["Clubs1"])
c2.update(dict(Name="👽", id="B"))
s = b["Spider Queen"]
s.update(dict(Name="🎇", id="C"))

Initialize the encounter. We can see who is involved and resource information by looking at the encounter's `PCs`, `enemies`, or `turn_order` properties.

In [5]:
e = Encounter(PCs=[c1, c2], Enemies=[s])
e.turn_order

[💀
 TC       : ♠️ 9 | pc.HP : 7/7
 Hand     :  02 | pc.PP : 2/2
 Deck     :  51 | pc.AP : 2/2
 Discards :  01 | RestC : 7/7,
 👽
 TC       : ♠️ 8 | pc.HP : 7/7
 Hand     :  02 | pc.PP : 2/2
 Deck     :  51 | pc.AP : 2/2
 Discards :  01 | RestC : 7/7,
 🎇
 TC       : ♥️ 4 | pc.HP : 8/8
 Hand     :  02 | pc.PP : 6/6
 Deck     :  51 | pc.AP : 2/2
 Discards :  01 | RestC : 8/8]

Next, we can simulate a couple rounds of combat. Here, each participant will choose a Power available to them (if sufficient PP) and apply it to an enemy at random. This does not yet cover buffing powers (e.g., Shield, Lend Aid) or mind control status effects (i.e., Charmed, Enthralled).

In [6]:
e.sim_round(3)

[40:02][INFO]: 💀 used 0/2 PP with Attack, Dual-Wield
[40:02][INFO]: 💀 used 0/2 PP with Attack, Weapon
[40:02][INFO]: 💀 wounded 🎇 by 1: AP 1/2, HP 8/8
[40:02][INFO]: 👽 used 0/2 PP with Drown
[40:02][INFO]: 🎇 is Suffocating: 1
[40:02][INFO]: 👽 used 1/2 PP with Attack, Charge
[40:02][INFO]: 🎇 used 1/6 PP with Attack, Mystic Entangle
[40:02][INFO]: 👽 resisted Entangled
[40:02][INFO]: 🎇 used 0/5 PP with Sear
[40:02][INFO]: 👽 is Burned: 1
[40:02][INFO]: 🎇 shakes off Suffocating
[40:02][INFO]: 💀 used 1/2 PP with Attack, Charge
[40:02][INFO]: 💀 wounded 🎇 by 2: AP 0/2, HP 7/8
[40:02][INFO]: 💀 used 0/1 PP with Attack, Dual-Wield
[40:02][INFO]: 👽 used 1/1 PP with Attack, Charge
[40:02][INFO]: 👽 wounded 🎇 by 2: AP 0/2, HP 5/8
[40:02][INFO]: 👽 remains Burned
[40:02][INFO]: 🎇 used 0/5 PP with Sear
[40:02][INFO]: 💀 is Burned: 1
[40:02][INFO]: 🎇 used 1/5 PP with Attack, Mystic Entangle
[40:02][INFO]: 💀 resisted Entangled
[40:02][INFO]: 🎇 wounded 💀 by 1: AP 1/2, HP 7/7
[40:02][INFO]: 💀 remains Burned
[

We can even simulate epic events.

In [7]:
e.sim_epic_event(successes_needed=3)

[40:02][INFO]: Party 1, GM 0 | 👽 Suited Hit
[40:02][INFO]: Party 2, GM 0 | 👽 Major Success
[40:02][INFO]: Party 3, GM 0 | 👽 Major Success
[40:02][INFO]: Party wins after 18 total cards drawn


Let's see how everyone is doing.

In [8]:
e.turn_order

[💀
 TC       : ♠️ 9 | pc.HP : 5/7
 Hand     :  03 | pc.PP : 0/2
 Deck     :  33 | pc.AP : 1/2
 Discards :  18 | RestC : 7/7,
 👽
 TC       : ♠️ 8 | pc.HP : 3/7
 Hand     :  03 | pc.PP : 0/2
 Deck     :  37 | pc.AP : 2/2
 Discards :  14 | RestC : 7/7,
 🎇
 TC       : ♥️ 4 | pc.HP : 5/8
 Hand     :  04 | pc.PP : 4/6
 Deck     :  46 | pc.AP : 0/2
 Discards :  04 | RestC : 8/8]

We can give some or all participants a rest.

In [9]:
e.sim_quick_rest(participants=e.PCs)  # If no participants specified, all

[40:02][INFO]: 💀 recovered 5 HP/PP/AP during Quick Rest
[40:02][INFO]: 👽 recovered 6 HP/PP/AP during Quick Rest
[40:02][INFO]: 🎇 recovered 7 HP/PP/AP during Quick Rest


In [10]:
e.turn_order

[💀
 TC       : ♠️ 5 | pc.HP : 7/7
 Hand     :  03 | pc.PP : 2/2
 Deck     :  41 | pc.AP : 2/2
 Discards :  10 | RestC : 5/7,
 👽
 TC       : ♦️ 3 | pc.HP : 7/7
 Hand     :  05 | pc.PP : 2/2
 Deck     :  43 | pc.AP : 2/2
 Discards :  06 | RestC : 2/7,
 🎇
 TC       : ♠️ 5 | pc.HP : 8/8
 Hand     :  04 | pc.PP : 6/6
 Deck     :  49 | pc.AP : 2/2
 Discards :  01 | RestC : 5/8]

Or a full rest.

In [11]:
e.sim_full_rest()

In [12]:
e.turn_order

[💀
 TC       : ♠️ 3 | pc.HP : 7/7
 Hand     :  03 | pc.PP : 2/2
 Deck     :  50 | pc.AP : 2/2
 Discards :  01 | RestC : 7/7,
 👽
 TC       : ♦️ 2 | pc.HP : 7/7
 Hand     :  05 | pc.PP : 2/2
 Deck     :  48 | pc.AP : 2/2
 Discards :  01 | RestC : 7/7,
 🎇
 TC       : ♦️ 2 | pc.HP : 8/8
 Hand     :  04 | pc.PP : 6/6
 Deck     :  49 | pc.AP : 2/2
 Discards :  01 | RestC : 8/8]